# Example of validation code

This shows an example of how to derive detection AJI/PQ scores from a model

In [10]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage.io
import glob
import albumentations as A
import random


# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN 
sys.path.append(ROOT_DIR)
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log
from mrcnn import stats_utils

import Monuseg

%matplotlib inline 
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

MONUSEG_DIR = os.path.join(ROOT_DIR, "datasets","monuseg")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Load dataset - See dataset Notebook
subset = "val"
dataset_dir = os.path.join(MONUSEG_DIR, subset)
image_dir = os.path.join(dataset_dir, "tissue_images")
image_ids = os.listdir(image_dir)
print("Found {} images in {}".format(len(image_ids), image_dir))


dataset = Monuseg.MonusegDataset()
dataset.add_class(source = "Monuseg", class_id =  1, class_name = "nucleus")

# Fill with the samples
i = 0
for _id in image_ids:
    name,_ = os.path.splitext(_id)
    if i <= len(image_ids):
        dataset.add_image(source = "Monuseg",
                                image_id = name,
                                path = os.path.join(image_dir, _id))
    i+=1    

    

dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

Found 14 images in C:\Users\User\GitHub\Mask_RCNN_Thesis\datasets\Monuseg_dist\val\tissue_images
Image Count: 14
Class Count: 2
  0. BG                                                
  1. nucleus                                           


In [13]:
# Load model
DEVICE = "/gpu:0"
TEST_MODE = "inference"
config = Monuseg_dist.MonusegDIST_H_InferenceConfig()

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)
weights_path = r"C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_5fold_coco_dist_h_tissue_img220210703T1151\mask_rcnn_monuseg_5fold_coco_dist_h_tissue_img2_0050.h5"

print("Loading weights ", weights_path)
# Create a folder to save detections at the dir of the model
model.load_weights(weights_path, by_name=True)
model_dir = os.path.split(weights_path)[0]

Loading weights  C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_5fold_coco_dist_h_tissue_img220210703T1151\mask_rcnn_monuseg_5fold_coco_dist_h_tissue_img2_0050.h5
Re-starting from epoch 50


Run detecttions, save results

In [5]:
Monuseg_dist.calc_and_save_detect(model,dataset,model_dir)

TCGA-2Z-A9J9-01A-01-TS1
Running detection...
Found 673 instances
Saving...
TCGA-44-2665-01B-06-BS6
Running detection...
Found 884 instances
Saving...
TCGA-69-7764-01A-01-TS1
Running detection...
Found 584 instances
Saving...
TCGA-A6-6782-01A-01-BS1
Running detection...
Found 522 instances
Saving...
TCGA-AC-A2FO-01A-01-TS1
Running detection...
Found 625 instances
Saving...
TCGA-AO-A0J2-01A-01-BSA
Running detection...
Found 446 instances
Saving...
TCGA-CU-A0YN-01A-02-BSB
Running detection...
Found 827 instances
Saving...
TCGA-EJ-A46H-01A-03-TSC
Running detection...
Found 561 instances
Saving...
TCGA-FG-A4MU-01B-01-TS1
Running detection...
Found 499 instances
Saving...
TCGA-GL-6846-01A-01-BS1
Running detection...
Found 409 instances
Saving...
TCGA-HC-7209-01A-01-TS1
Running detection...
Found 380 instances
Saving...
TCGA-HT-8564-01Z-00-DX1
Running detection...
Found 282 instances
Saving...
TCGA-IZ-8196-01A-01-BS1
Running detection...
Found 631 instances
Saving...
TCGA-ZF-A9R5-01A-01-TS1
R

Load detections

In [6]:
pred_mask_flat_dict = {}
detection_dir = os.path.join(model_dir, "Detections")
for _id in dataset.image_ids:
    n = dataset.image_info[_id]["id"]
    m = np.load(os.path.join(detection_dir, n + "_label.npy"))
    print("Loaded: {}, Detections: {}".format(n, np.max(m)))
    pred_mask_flat_dict[n] = m
    

Loaded: TCGA-2Z-A9J9-01A-01-TS1, Detections: 673
Loaded: TCGA-44-2665-01B-06-BS6, Detections: 884
Loaded: TCGA-69-7764-01A-01-TS1, Detections: 584
Loaded: TCGA-A6-6782-01A-01-BS1, Detections: 522
Loaded: TCGA-AC-A2FO-01A-01-TS1, Detections: 625
Loaded: TCGA-AO-A0J2-01A-01-BSA, Detections: 446
Loaded: TCGA-CU-A0YN-01A-02-BSB, Detections: 827
Loaded: TCGA-EJ-A46H-01A-03-TSC, Detections: 561
Loaded: TCGA-FG-A4MU-01B-01-TS1, Detections: 499
Loaded: TCGA-GL-6846-01A-01-BS1, Detections: 409
Loaded: TCGA-HC-7209-01A-01-TS1, Detections: 380
Loaded: TCGA-HT-8564-01Z-00-DX1, Detections: 282
Loaded: TCGA-IZ-8196-01A-01-BS1, Detections: 631
Loaded: TCGA-ZF-A9R5-01A-01-TS1, Detections: 538


Load GT data

In [7]:
gt_mask_flat_dict = {}
for _id in dataset.image_ids:   
    info = dataset.image_info[_id]
    n = info["id"]
    m = dataset.load_labelled_mask(_id)
    print("Loaded: {}, Detections: {}".format(n, np.max(m)))
    gt_mask_flat_dict[n] = m


Loaded: TCGA-2Z-A9J9-01A-01-TS1, Detections: 575
Loaded: TCGA-44-2665-01B-06-BS6, Detections: 818
Loaded: TCGA-69-7764-01A-01-TS1, Detections: 441
Loaded: TCGA-A6-6782-01A-01-BS1, Detections: 441
Loaded: TCGA-AC-A2FO-01A-01-TS1, Detections: 521
Loaded: TCGA-AO-A0J2-01A-01-BSA, Detections: 356
Loaded: TCGA-CU-A0YN-01A-02-BSB, Detections: 677
Loaded: TCGA-EJ-A46H-01A-03-TSC, Detections: 466
Loaded: TCGA-FG-A4MU-01B-01-TS1, Detections: 465
Loaded: TCGA-GL-6846-01A-01-BS1, Detections: 355
Loaded: TCGA-HC-7209-01A-01-TS1, Detections: 325
Loaded: TCGA-HT-8564-01Z-00-DX1, Detections: 249
Loaded: TCGA-IZ-8196-01A-01-BS1, Detections: 533
Loaded: TCGA-ZF-A9R5-01A-01-TS1, Detections: 475


Calculate Scores, save them

In [8]:
# Set up to txt
save_path = os.path.join(model_dir, "stats.txt")
file = open(save_path, "a") 
file.write("NAME\tAJI\tPQ\tDQ\tSQ\n")
print("Logging to {}".format(save_path))
avg_aji = 0
avg_pq = 0
avg_dice1 = 0
avg_dice2 = 0

for n in pred_mask_flat_dict.keys():
    
    print("Running calculations on: {}".format(n))
    
    pred_remap = stats_utils.remap_label(pred_mask_flat_dict[n])
    gt_remap = stats_utils.remap_label(gt_mask_flat_dict[n])
    
    aji = stats_utils.get_fast_aji(gt_remap, pred_remap)
    pq = stats_utils.get_fast_pq(gt_remap, pred_remap)
    dice1 = stats_utils.get_dice_1(gt_remap, pred_remap)
    dice2 = stats_utils.get_fast_dice_2(gt_remap, pred_remap)

    
    avg_aji += aji
    avg_pq += pq[0][2]
    avg_dice1 += dice1
    avg_dice2 += dice2
    
    print("AJI: {}\tPQ:{}".format(aji, pq[0][2]))
#   Write the results in a txt:
    file.write(n + "\t" + str(aji) + "\t" + str(pq[0][2]) + "\t" + str(pq[0][0]) + "\t" + str(pq[0][1]) +  
               "\t" + str(dice1) + "\t" + str(dice2) + "\n")

avg_aji = avg_aji/len(pred_mask_flat_dict.keys())   
avg_pq = avg_pq/len(pred_mask_flat_dict.keys())
avg_dice1 = avg_dice1/len(pred_mask_flat_dict.keys())
avg_dice2 = avg_dice1/len(pred_mask_flat_dict.keys())
print("Mean AJI: " + str(avg_aji) + " Mean PQ: " + str(avg_pq))
file.write("Mean AJI: " + str(avg_aji) + " Mean PQ: " + str(avg_pq))
file.close()

Logging to C:\Users\User\GitHub\Mask_RCNN_Thesis\logs\monuseg_5fold_coco_dist_h_tissue_img220210703T1151\stats.txt
Running calculations on: TCGA-2Z-A9J9-01A-01-TS1
AJI: 0.5541808220165629	PQ:0.5469384399352768
Running calculations on: TCGA-44-2665-01B-06-BS6
AJI: 0.6523957181761804	PQ:0.6740675001645897
Running calculations on: TCGA-69-7764-01A-01-TS1
AJI: 0.6145808201116635	PQ:0.6183868037206715
Running calculations on: TCGA-A6-6782-01A-01-BS1
AJI: 0.560158956505355	PQ:0.5696642180551061
Running calculations on: TCGA-AC-A2FO-01A-01-TS1
AJI: 0.5622277009431843	PQ:0.5587043645318208
Running calculations on: TCGA-AO-A0J2-01A-01-BSA
AJI: 0.5534361125413457	PQ:0.5251407445321958
Running calculations on: TCGA-CU-A0YN-01A-02-BSB
AJI: 0.5702510160692751	PQ:0.6122307718251538
Running calculations on: TCGA-EJ-A46H-01A-03-TSC
AJI: 0.5868723921894894	PQ:0.6022960076475962
Running calculations on: TCGA-FG-A4MU-01B-01-TS1
AJI: 0.6463256098493538	PQ:0.6206396853284347
Running calculations on: TCGA-G